In [123]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas_ta as ta # pip install pandas-ta
import yfinance as yf # pip install yfinance

In [124]:
listings = ['MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIA']

In [171]:
df = yf.download(tickers=listings, period='10y', interval='1d', auto_adjust=False)

[*********************100%***********************]  31 of 31 completed


In [172]:
# stack and unstack indexed columns
df = df.stack(level=0).unstack(level=1)

# set each column name to TICKER - VALUE
df.columns = df.columns.map(lambda x: f'{x[0]} - {x[1]}')

# create date column from index
df = df.reset_index()
df = df.rename(columns={'index': 'Date'})

In [173]:
for ticker in listings:
    col_name = f'{ticker} - Dividend'
    div_series = yf.Ticker(ticker).dividends
    series_df = div_series.to_frame(name=col_name)
    series_df.index = series_df.index.tz_convert(None).floor('D')
    df = pd.merge(df, series_df, left_on='Date', right_index=True, how='left')
    df[col_name] = df[col_name].fillna(0)

In [174]:
df[df['DIA - Dividend'] != 0]

,Date,AAPL - Adj Close,AAPL - Close,AAPL - High,AAPL - Low,AAPL - Open,AAPL - Volume,AMGN - Adj Close,AMGN - Close,AMGN - High,...,NKE - Dividend,PG - Dividend,CRM - Dividend,TRV - Dividend,UNH - Dividend,VZ - Dividend,V - Dividend,WBA - Dividend,WMT - Dividend,DIA - Dividend
14,2013-06-21,12.821410,14.767857,15.000000,14.575000,14.946071,481118400.0,75.239700,98.279999,99.120003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.304
33,2013-07-19,13.176438,15.176786,15.499286,15.155357,15.467857,268721600.0,83.745155,109.389999,109.680000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.206
53,2013-08-16,15.678610,17.940357,17.962143,17.816429,17.862499,362306000.0,80.666908,104.910004,106.800003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.315,0.0,0.383
77,2013-09-20,14.588691,16.693214,17.091070,16.642857,17.071428,699302800.0,89.847740,116.849998,117.610001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.304
97,2013-10-18,15.883359,18.174643,18.187857,18.061071,18.071072,290542000.0,88.363747,114.919998,116.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,2023-01-20,137.469666,137.869995,138.020004,134.220001,135.279999,80223600.0,258.499847,263.239990,263.839996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.280
2446,2023-02-17,152.339294,152.550003,153.000000,150.850006,152.350006,59144100.0,238.281754,240.529999,241.720001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.552
2465,2023-03-17,154.785904,155.000000,156.740005,154.279999,156.080002,98944600.0,227.642136,229.789993,233.809998,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.858
2489,2023-04-21,164.792068,165.020004,166.449997,164.490005,165.050003,58337300.0,241.580643,243.860001,243.919998,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.184


In [175]:
df.to_csv('dija_w_divs.csv', index=False)

In [176]:
df2 = pd.read_csv('dija_w_divs.csv')

In [177]:
df2.head()

,Date,AAPL - Adj Close,AAPL - Close,AAPL - High,AAPL - Low,AAPL - Open,AAPL - Volume,AMGN - Adj Close,AMGN - Close,AMGN - High,...,NKE - Dividend,PG - Dividend,CRM - Dividend,TRV - Dividend,UNH - Dividend,VZ - Dividend,V - Dividend,WBA - Dividend,WMT - Dividend,DIA - Dividend
0,2013-06-03,13.975490,16.097143,16.155714,15.802857,16.097500,372352400.0,75.890419,99.129997,101.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-06-04,13.931776,16.046785,16.229643,15.978214,16.186428,292728800.0,74.481819,97.290001,100.360001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-06-05,13.801541,15.896786,16.097143,15.846786,15.916071,290589600.0,72.422432,94.599998,98.330002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-06-06,13.595348,15.659286,15.964286,15.501786,15.909643,416934000.0,74.634933,97.489998,97.529999,...,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-06-07,13.699219,15.778929,15.830000,15.456071,15.589286,404535600.0,75.645454,98.809998,100.260002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
